# Descarga, limpieza y procesamiento de capas vectoriales

In [1]:
import geopandas as gpd
import requests
import zipfile
import io
import os

In [2]:
# ===============================
# WFS: CICLORUTAS DE CALI (IDESC)
# ===============================

# 1. Definir URL del servicio
url = "https://ws-idesc.cali.gov.co/geoserver/pot_2014/ows"
params = {
    "service": "WFS",
    "version": "1.0.0",
    "request": "GetFeature",
    "typeName": "pot_2014:mov_jerarquizacion_ciclorutas",
    "outputFormat": "SHAPE-ZIP"
}

# 2. Hacer la solicitud HTTP
response = requests.get(url, params=params)

# 3. Descomprimir en memoria
zip_file = zipfile.ZipFile(io.BytesIO(response.content))
zip_file.extractall("ciclorutas_cali")

# 4. Cargar con GeoPandas
gdf = gpd.read_file("ciclorutas_cali/mov_jerarquizacion_ciclorutas.shp")

Index(['condicion', 'prioridad', 'nombre', 'id_condici', 'geometry'], dtype='object')
   condicion prioridad                                             nombre  \
0  Propuesta     Media  Calle 60 y Carrera 99, entre Calle 25 y Vía Pu...   
1  Propuesta      Alta          Carrera 122, entre Calle 1 a la vía Pance   
2  Existente     Media               Carrera 25, entre Carreras 26G y 28F   
3  Propuesta     Media                  Avenida 3N entre Calles 18 y 23CN   
4  Propuesta     Media                  Avenida 3N entre Calles 23CN y 25   

   id_condici                                           geometry  
0           2  LINESTRING (1063326.839 860008.899, 1063581.66...  
1           2  LINESTRING (1058994.76 861347.819, 1058858.487...  
2           1  LINESTRING (1068035.696 870572.082, 1068006.06...  
3           2  LINESTRING (1060877.393 874097.009, 1061141.61...  
4           2  LINESTRING (1061542.824 875076.141, 1061535.80...  


In [3]:
gdf

,condicion,prioridad,nombre,id_condici,geometry
0,Propuesta,Media,"Calle 60 y Carrera 99, entre Calle 25 y Vía Pu...",2,"LINESTRING (1063326.839 860008.899, 1063581.66..."
1,Propuesta,Alta,"Carrera 122, entre Calle 1 a la vía Pance",2,"LINESTRING (1058994.76 861347.819, 1058858.487..."
2,Existente,Media,"Carrera 25, entre Carreras 26G y 28F",1,"LINESTRING (1068035.696 870572.082, 1068006.06..."
3,Propuesta,Media,Avenida 3N entre Calles 18 y 23CN,2,"LINESTRING (1060877.393 874097.009, 1061141.61..."
4,Propuesta,Media,Avenida 3N entre Calles 23CN y 25,2,"LINESTRING (1061542.824 875076.141, 1061535.80..."
...,...,...,...,...,...
98,Propuesta,Media,"Carrera 66, entre Calles 9 y 25",2,"LINESTRING (1059234.98 867531.493, 1059974.406..."
99,Propuesta,Media,"Avenida 3N, entre Calles 55N y 70",2,"LINESTRING (1062793.542 877208.009, 1062889.25..."
100,Propuesta,Media,"Calle 34N, entre Avenidas 2Bis y 3N",2,"LINESTRING (1061807.29 875628.813, 1061502.733..."
101,Propuesta,Alta,"Carrera 4, entre Calles 5 y 25",2,"LINESTRING (1061944.924 874193.215, 1061407.38..."


In [ ]:
# 6. Limpieza básica
gdf = gdf.rename(columns={
    'nombre': 'nombre_via',
    'JERARQUIA': 'jerarquia',
    'LONGITUD_M': 'longitud_m',
})
gdf = gdf.to_crs("EPSG:4326")  # reproyectar a WGS84

# 7. Exportar a GeoJSON y CSV
gdf.to_file("ciclorutas_cali.geojson", driver="GeoJSON")
gdf[['nombre_via', 'jerarquia', 'longitud_m', 'geometry']].to_csv("ciclorutas_cali.csv", index=False)

print("✅ Archivos exportados: ciclorutas_cali.geojson y ciclorutas_cali.csv")